<h2>Ajax 数据爬取</h2>

在用 `requests` 抓取页面的时候，得到的结果可能和在浏览器中看到的不一样：在浏览器中可以看到正常显示的页面数据，但是使用 `requests` 得到的结果并没有。

这是因为 `requests` 获取的都是原始的 HTML 文档，而浏览器中的页面则是经过 JavaScript 处理数据后生成的结果，这些数据的来源有多种，可能是通过 Ajax 加载的，可能是包含在 HTML 文档中的，也可能是经过 JavaScript 和特定算法计算后生成的。

对于第一种情况，数据加载是一种异步加载方式，原始的页面最初不会包含某些数据，原始页面加载完后，会再向服务器请求某个接口获取数据，然后数据才被处理从而呈现到网页上，这其实就是发送了一个 Ajax 请求。

网页的原始 HTML 文档不会包含任何数据，数据都是通过 Ajax 统一加载后再呈现出来的，这样在 Web 开发上可以做到前后端分离，而且降低服务器直接渲染页面带来的压力。

如果遇到这样的页面，直接利用 `requests` 等库来抓取原始页面，是无法获取到有效数据的，这时需要分析网页后台向接口发送的 Ajax 请求，用 `requests` 来模拟 Ajax 请求。

**什么是 Ajax**

Ajax，Asynchronous JavaScript and XML，即异步的 JavaScript 和 XML。

它不是一门编程语言，而是利用 JavaScript 在保证页面不被刷新、页面链接不改变的情况下与服务器交换数据并更新部分网页的技术。

对于传统的网页，如果想更新其内容，那么必须要刷新整个页面，但有了 Ajax，便可以在页面不被全部刷新的情况下更新其内容。在这个过程中，页面实际上是在后台与服务器进行了数据交互，获取到数据之后，再利用 JavaScript 改变网页内容。

可以到 W3School 上体验几个 Demo 来感受一下：https://www.w3school.com.cn/ajax/ajax_xmlhttprequest_send.asp

**实例引入**

浏览网页的时候，会发现很多网页都有“下滑查看更多”的选项。

以作者微博的主页为例：https://m.weibo.cn/u/2830678474 切换到微博页面，一直下滑，可以发现下滑几个微博之后，再向下就没有了，转而会出现一个加载的动画，不一会儿下方就继续出现了新的微博内容，这个过程其实就是 Ajax 加载的过程。

> 页面其实并没有整个刷新，也就意味着页面的链接没有变化，但是网页中却多了新内容，也就是后面刷出来的新微博。这就是通过 Ajax 获取新数据并呈现的过程。

**Ajax基本原理**

发送 Ajax 请求到网页更新的这个过程可以简单分为以下 3 步：

- 发送请求（JavaScript 可以实现页面的各种交互功能）实际上执行了如下代码：
    ```javascript
    var xmlhttp;
    if (window.XMLHttpRequest) {
        //code for IE7+, Firefox, Chrome, Opera, Safari
        xmlhttp=new XMLHttpRequest();} else {//code for IE6, IE5
        xmlhttp=new ActiveXObject("Microsoft.XMLHTTP");
    }
    xmlhttp.onreadystatechange=function() {if (xmlhttp.readyState==4 && xmlhttp.status==200) {document.getElementById("myDiv").innerHTML=xmlhttp.responseText;
        }
    }
    xmlhttp.open("POST","/ajax/",true);
    xmlhttp.send();
    ```
  > 这是 JavaScript 对 Ajax 最底层的实现，实际上就是新建了 `XMLHttpRequest` 对象，然后调用 `onreadystatechange` 属性设置了监听，
  > 
  > 然后调用 `open()` 和 `send()` 方法向某个链接（也就是服务器）发送了请求。
  > 
  > 前面用 Python 实现请求发送之后，可以得到响应结果，但这里请求的发送变成 JavaScript 来完成。
  > 
  > 由于设置了监听，所以当服务器返回响应时，`onreadystatechange` 对应的方法便会被触发，然后在这个方法里面解析响应内容即可。
- 解析内容
  得到响应之后，`onreadystatechange` 属性对应的方法便会被触发，此时利用 `xmlhttp` 的 `responseText` 属性便可取到响应内容。
  
  这类似于 Python 中利用 `requests` 向服务器发起请求，然后得到响应的过程。
  
  那么返回内容可能是 HTML，可能是 JSON，接下来只需要在方法中用 JavaScript 进一步处理即可。比如，如果是 JSON 的话，可以进行解析和转化。
- 渲染网页
  **JavaScript 有改变网页内容的能力**
  
  可以通过 `document.getElementById().innerHTML` 这样的操作，便可以对某个元素内的源代码进行更改，这样网页显示的内容就改变了，这样的操作也被称作 **DOM 操作**，即对 Document 网页文档进行操作，如更改、删除等。
  

以上 3 个步骤（整个请求、解析和渲染的过程）由 JavaScript 完成。

**Ajax 分析方法**

以前面的微博为例

1. 查看请求

    借助浏览器的开发者工具（以 Chrome 浏览器为例）
  
    首先，用 Chrome 浏览器打开微博的链接 https://m.weibo.cn/u/2830678474 随后在页面中点击鼠标右键，从弹出的快捷菜单中选择 “检查” 选项，此时便会弹出开发者工具。

    此时在 `Elements` 选项卡中便会观察到网页的源代码，切换到 `Network` 选项卡（在页面加载过程中浏览器与服务器之间发送请求和接收响应的所有记录）

    Ajax 其实有其特殊的请求类型，它叫作 `xhr`。名称以 `getIndex` 开头的请求，其 Type 为 `xhr`，这就是一个 Ajax 请求。

    ![](6-1.png)

    所以说，我们看到的微博页面的真实数据并不是最原始的页面返回的，而是后来执行 JavaScript 后再次向后台发送了 Ajax 请求，浏览器拿到数据后再进一步渲染出来的。

2. 过滤请求

    利用 Chrome 开发者工具的筛选功能筛选出所有的 Ajax 请求。在请求的上方有一层筛选栏，直接点击 `Fetch/XHR`，此时在下方显示的所有请求便都是 Ajax 请求

    接下来，不断滑动页面，可以看到页面底部有一条条新的微博被刷出，而开发者工具下方也一个个地出现 Ajax 请求，这样就可以捕获到所有的 Ajax 请求。

    随意点开一个条目，Headers 都可以清楚地看到其 `Request URL`、`Request Headers`、`Response Headers`、`Response Body` 等内容。

到现在为止，已经可以分析出 Ajax 请求的一些详细信息了，接下来只需要用程序模拟这些 Ajax 请求，就可以轻松提取需要的信息。

**Ajax 结果提取**

仍然以微博为例，接下来用 Python 来模拟这些 Ajax 请求。

1. 分析请求

    打开 Ajax 的 XHR 过滤器，然后一直滑动页面以加载新的微博内容。可以看到，会不断有 Ajax 请求发出。

    选定其中一个请求，分析它的参数信息。以链接 https://m.weibo.cn/api/container/getIndex?type=uid&value=2145291155&containerid=1076032145291155&page=2 为例：
    
    请求的参数有四个：`type、value、containerid、page`。

    > 随后再看看其他请求，可以发现，它们的 `type、value` 和 `containerid` 始终如一。
    > 
    > `type` 始终为 uid，value 的值就是页面链接中的数字，其实这就是用户的 id。
    > 
    > `containerid` 是 107603 加上用户 id。
    > 
    > 改变的值就是 `page`，很明显这个参数是用来控制分页的，page=1 代表第一页，page=2 代表第二页，以此类推。

2. 分析响应

    随后，观察这个请求的响应内容（JSON 格式）：
    ![](6-2.png)
    
    > 最关键的两部分信息就是 `cardlistInfo` 和 `cards`：
    > 
    > - 前者包含一个比较重要的信息 `total` —— 微博的总数量，可以根据这个数字来估算分页数；
    > 
    > - 后者则是一个列表，它包含 10 个元素。
    > （重要的字段 `mblog`。展开它，可以发现它包含的正是微博的一些信息，
    >
    > 比如 `attitudes_count`（赞数目）、`comments_count`（评论数目）、`reposts_count`（转发数目）、`created_at`（发布时间）、`text`（微博正文）等，而且它们都是一些格式化的内容。

这样请求一个接口，就可以得到 10 条微博，而且请求时只需要改变 `page` 参数即可。

In [1]:
from urllib.parse import urlencode  
import requests
import scrapy

base_url = 'https://m.weibo.cn/api/container/getIndex?'  

headers = {  
    'Host': 'm.weibo.cn',  
    'Referer': 'https://m.weibo.cn/u/2830678474',  
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',  
    'X-Requested-With': 'XMLHttpRequest',  
}  

def get_page(page):  
    params = {  
        'type': 'uid',  
        'value': '2830678474',  
        'containerid': '1076032830678474',  
        'page': page  
    }  
    url = base_url + urlencode(params)  
    try:  
        response = requests.get(url, headers=headers)  
        if response.status_code == 200:  
            return response.json()  
    except requests.ConnectionError as e:  
        print('Error', e.args)
        
# 定义一个解析方法，用来从结果中提取想要的信息，
# 比如这次想保存微博的 id、正文、赞数、评论数和转发数这几个内容，
# 那么可以先遍历 cards，然后获取 mblog 中的各个信息，赋值为一个新的字典返回即可

from pyquery import PyQuery as pq  

def parse_page(json):  
    if json:  
        items = json.get('data').get('cards')  
        for item in items:  
            item = item.get('mblog')  
            weibo = {}  
            weibo['id'] = item.get('id')  
            weibo['text'] = pq(item.get('text')).text()  
            weibo['attitudes'] = item.get('attitudes_count')  
            weibo['comments'] = item.get('comments_count')  
            weibo['reposts'] = item.get('reposts_count')  
            yield weibo


# 遍历10页       
for page in range(1, 11):  
        json = get_page(page)  
        results = parse_page(json)  
        for result in results:  
            print(result)

{'id': '5016437486518851', 'text': '转发微博', 'attitudes': 6, 'comments': 0, 'reposts': 2}
{'id': '5015254340209061', 'text': '北京 崔庆才丨静觅的微博视频', 'attitudes': 2, 'comments': 2, 'reposts': 0}
{'id': '5015019637178593', 'text': '实在是高！', 'attitudes': 2, 'comments': 1, 'reposts': 1}
{'id': '5012880691299523', 'text': '', 'attitudes': 1, 'comments': 0, 'reposts': 1}
{'id': '5009817909330018', 'text': '一只又乖又好看的招财猫 北京', 'attitudes': 8, 'comments': 2, 'reposts': 0}
{'id': '5009815632347393', 'text': '猫咪狂想曲 北京 崔庆才丨静觅的微博视频', 'attitudes': 5, 'comments': 0, 'reposts': 0}
{'id': '5007268665626463', 'text': '面向猫猫编程 崔庆才丨静觅的微博视频', 'attitudes': 9, 'comments': 1, 'reposts': 0}
{'id': '5006866248110639', 'text': '转发微博', 'attitudes': 0, 'comments': 0, 'reposts': 2}
{'id': '5006408536033990', 'text': '转发微博', 'attitudes': 0, 'comments': 0, 'reposts': 0}
{'id': '5005270026030012', 'text': '//@王尼玛:', 'attitudes': 1, 'comments': 1, 'reposts': 0}
{'id': '5005042991499580', 'text': '//@金山://@老刀99:能有工牌戴都不错了，那些再老一点的牛不仅

另外，还可以加一个方法将结果**保存到 MongoDB** 数据库：

```python
from pymongo import MongoClient  

client = MongoClient()  
db = client['weibo']  
collection = db['weibo']

def save_to_mongo(result):  
    if collection.insert(result):  
        print('Saved to Mongo')
```

<h2>分析 Ajax 爬取 </h2>

In [ ]:
import requests  
from urllib.parse import urlencode
from pyquery import PyQuery as pq


class WeiboSpider:
    def __init__(self, user_id):
        self.container = '107603'  # 固定
        self.user_id = user_id
        self.headers = {
            'Host': 'm.weibo.cn',
            'Referer': 'https://m.weibo.cn/u/' + user_id,
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
            'X-Requested-With': 'XMLHttpRequest',
        }
        self.base_url = 'https://m.weibo.cn/api/container/getIndex?'
        
    def get_page(self, page):
        """
        :param page: 
        :return: 
        """
        params = {  
        'type': 'uid',  
        'value': self.user_id,  
        'containerid': self.container + self.user_id,
        'page': page  
        }  
        url = self.base_url  + urlencode(params)  
        try:  
            response = requests.get(url)  
            if response.status_code == 200:  
                return response.json()  
        except requests.ConnectionError:  
            return None

    def parse_page(self, json):
        """
        :param json:
        :return: 
        """
        if json:  
            items = json.get('data').get('cards')  
            for item in items:  
                item = item.get('mblog')  
                weibo = {}  
                weibo['id'] = item.get('id')  
                weibo['text'] = pq(item.get('text')).text()  
                weibo['attitudes'] = item.get('attitudes_count')  
                weibo['comments'] = item.get('comments_count')  
                weibo['reposts'] = item.get('reposts_count')
                pics = item.get('pics')  # list & None
                if pics is not None:
                    weibo['pics'] = [pic.get('url') for pic in pics]
                else:
                    weibo['pics'] = None
                yield weibo


# user_id = '1669879400'  # 迪丽热巴
user_id = '2303490360'  # 郑合惠子
weibo = WeiboSpider(user_id)

# 遍历10条     
for page in range(1, 11):  
        json = weibo.get_page(page) 
        results = weibo.parse_page(json)  
        for result in results:  
            print(result)